<a href="https://colab.research.google.com/github/VAC10/C-Application/blob/main/Turkcell_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# collab bağlantısı

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import cv2

import urllib

import itertools

import numpy as np

import pandas as pd

import seaborn as sns 

import random, os, glob

from imutils import paths 

import matplotlib.pyplot as plt

from sklearn.utils import shuffle 

from urllib.request import urlopen

#Warningleri kapatmak için kullanilmaktadir.
import warnings
warnings.filterwarnings('ignore')

#Model değerlendirme için kullanilacak olan kütüphaneler 
from sklearn.metrics import confusion_matrix, classification_report

#Model için kullanılacak olan kütüphaneler 
import tensorflow as tf

from tensorflow.keras.models import Sequential 
from tensorflow.keras.preprocessing import image

from tensorflow.keras.utils import to_categorical

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping 
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D, Dense, Dropout, SpatialDropout2D

from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

2)VERİ SETİNİN OKUNMASI

>a)veri seti adresinin collaba gonderilmesi




In [ ]:
from google.colab import drive
drive.mount('/content/drive')



KeyboardInterrupt: ignored

In [ ]:
dir_path ='/content/drive/MyDrive/Garbage classification/Garbage classification' # bu veri setinin yer aldığı pathin tutulması..

In [ ]:
#target size ve label etiket değerlerinin belirlenmesi
target_size=(224,224)
waste_labels ={'cardboard': 0,'glass': 1,'metal': 2, 'paper': 3, 'plastic': 4,'trash': 5} # her bir klasorun ismini kullanarak her bir görsele etiketleme yaptık ve endcode ettik

In [ ]:
def load_datasets(path):
    x=[]
    labels=[]
  

    """

    Görsellerin bulunduğu dizindeki görüntüyü okuyup etiketlerini oluşturur.

    Parametreler:

    path: Görsellerin bulunduğu dizini ifade eder.

    Return:

    x: Görüntülere ait matris bilgilerini tutar.

    labels: Görüntünün ait olduğu sınıif bilgisini tutan liste.
    """


    # Gönderdiğimiz pathdeki görüntüleri listeleyip sıralamaktadır. 
    image_paths=sorted(list(paths.list_images(path)))

    for image_path in image_paths: # Belirtilen pathdeki görüntüler openCV kütüphanesi ile okunmaktadır.
        img=cv2.imread(image_path)

      # Okunan görüntüler başlangıçta belirlenen target size'a göre yeniden ölçeklendirilir. 
        img= cv2.resize(img, target_size)

      # ölçeklendirilen görüntüler x listesine eklenir.
        x.append(img)
      # Her bir path '/' ifadesi ile ayrıldığında dönen listenin sondan ikinci elemanı labelı temsil etmektedir.
        label=image_path.split(os.path.sep)[-2]
  # Yakalanan labelların sayısal değer karşılıklarının olduğu waste labels sözlüğü içerisinden gönderilen key # değerine karşılik value,değeri alınarak label oluşturulur.
        labels.append(waste_labels[label])

        # Veri seti random bir şekilde karıştırılır.
    x, labels=shuffle(x, labels, random_state=42)

            # Boyut ve Sinıf Bilgisi raporlanmaktadır. 
    print (f"X boyutu: {np.array(x).shape}") 
    print(f"Label sinıf sayıs1: {len(np.unique(labels))} Gözlem sayısı: {len(labels)}")

    return x, labels




In [ ]:
x,labels=load_datasets(dir_path)

In [ ]:
def visualize_img(image_batch, label_batch):
    """
    Veri seti içerisinden görüntü görselleştirir.

    Parametreler:

    image batch: Görüntülere ait matris bilgilerini tutar.

    label batch: Görüntünün ait olduğu sınıf bilgisini tutan liste.
    """

    plt.figure(figsize=(10,10)) 
    for n in range(10):

        ax =plt.subplot (5,5,n+1)
        plt.imshow(image_batch[n])

        plt.title(np.array(list(waste_labels.keys()))[to_categorical(labels, num_classes=6)[n]==1][0].title()) 
        plt.axis('off')



In [ ]:
visualize_img(x,labels)

3)VERİYİ HAZIRLAMA


In [ ]:
#train veri seti için bir generator tanımlıyoruz
train=ImageDataGenerator(horizontal_flip=True,# görüntüyü rastgele yatay olarak çeviriir
                         vertical_flip=True,# görüntüyü rastgele dikey olarak çeviriir
                         validation_split=0.1,# test veri setinin oranı
                         rescale=1./255,#verileri sağlanan değerlere göre yeniden ölçeklendirir 
                         shear_range=0.1, #görüntüye eğim verir
                         zoom_range=0.1,
                         width_shift_range=0.1,
                         height_shift_range=0.1) # verisetimizde bulunan görselleri görüntü çeşitlendirme teknikleri uygulayark gerçek dünya özelliklerini ekleyecez
                         
##test veri seti için bir generator tanımlıyoruz 

test=ImageDataGenerator( 
                         rescale=1/255,
                         validation_split=0.1)
                         




In [ ]:
train_generator=train.flow_from_directory(directory=dir_path,
                                        target_size=(target_size),                                        
                                        class_mode='categorical',
                                        subset='training')

test_generator=test.flow_from_directory(directory=dir_path,
                                        target_size=(target_size), 
                                        batch_size=251,
                                        class_mode='categorical',
                                        subset='validation')


In [ ]:
"""
flattena olan kadar kısımda ben resmimin öz niteliklerini
çıkarıyorum. conv ile filtreyle üzerinde gezerek resimlerimi tam olarak algılamaya
çalışıyorum pooling ile boyutunu düşürüyorum.
flattendan sonra ise öğrenme kısmımı oluşturuyorum 
yani nöromlarımı oluşturuyorum ve dropout yaparak overfittingin önüne geçiyorum.
"""
model=Sequential()
model.add(Conv2D(filters=32,kernel_size=(3,3),padding="same",input_shape=(224,224,3),activation="relu"))
model.add(MaxPooling2D(pool_size=2,strides=(2,2)))

model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same",input_shape=(224,224,3),activation="relu"))
model.add(MaxPooling2D(pool_size=2,strides=(2,2)))

model.add(Conv2D(filters=32,kernel_size=(3,3),padding="same",input_shape=(224,224,3),activation="relu"))
model.add(MaxPooling2D(pool_size=2,strides=(2,2)))

model.add(Flatten())#yapay sinir ağları giriş verilerini tek boyutlu diziden alır
# ama pooling ve evrişim katmanında veriler matris olarak gelir. flatten ile biz bu katmanlardan gelen verileri tek boyutlu diziye çeviririz. 

model.add(Dense(units=64,activation="relu"))
model.add(Dropout(rate=0.2))

model.add(Dense(units=32,activation="relu"))
model.add(Dropout(rate=0.2))

model.add(Dense(units=6,activation="softmax")) # 6 sınıflı bir sınıflandırma problemimiz var 
#ikiden fazla sınıflandırma olduğundan dolayı softmax kullanırım







MODEL ÖZETİ


In [ ]:
model.summary()

OPTİMİZASYON VE DEĞERLENDİRME METRİKLERİNİN AYARLANMASI

In [ ]:
model.compile(loss="categorical_crossentropy",# ikiden fazla sınıf olduğundan categoircal kullandım 2 tane olaydı binary cross kullanılırdı
              optimizer="adam",
              metrics=[tf.keras.metrics.Precision(),tf.keras.metrics.Recall(),"acc"]
              )

In [ ]:
callbacks=[EarlyStopping(monitor="val_loss",patience=50,verbose=1,mode="min"),# model ezberlemeye başladığı an durdur(50 epochtan sonra) early stopping
           #monitorde ise modelin ezberlemeye başladığını hangi metrikle anlayacağız demektir
           ModelCheckpoint(filepath="mymodel.h5",monitor="val_loss",mode="min",save_best_only=True,save_weights_only=False,verbose=1) # iterasyonda yakalanan en iyi modelin kaydedilmesi
]

MODELİN EĞİTİLMESİ


In [ ]:
history=model.fit_generator(generator=train_generator,
                            epochs=100,
                            validation_data=test_generator,
                            callbacks=callbacks,
                            workers=4, # paralel olarak gruplar hesaplanır(donanımsal parametredir)
                            steps_per_epoch=2275/31,# her epochta 71 adımda model eğitimi gerçekleşecek
                            validation_steps=252/31)

